<a href="https://colab.research.google.com/github/samitha278/miniVGG/blob/main/build_vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import torch
import torch.nn.functional as F

## Simple Convolution function

In [7]:
def Convolution(image,kernel):

  x,y = image.shape

  a,b = kernel.shape

  feature_map = []

  for i in range(y-b):

    row = []

    for j in range(x-a):

      temp = (image[i:i+b,j:j+a] * kernel).sum(dim=(0,1))

      row.append(temp.item())

    feature_map.append(row)

  return torch.tensor(feature_map)

In [8]:
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)


feature_map = Convolution(image,kernel)


In [9]:
feature_map.shape

torch.Size([29, 29])

## Convolution with stride

In [10]:
def Convolution(image,kernel,stride=(1,1)):

  x,y = image.shape
  a,b = kernel.shape

  r,c = stride

  feature_map = []

  for i in range(0,y-b,c):

    row = []

    for j in range(0,x-a,r):

      temp = (image[i:i+b,j:j+a] * kernel).sum(dim=(0,1))

      row.append(temp.item())

    feature_map.append(row)

  return torch.tensor(feature_map)

In [11]:
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)


feature_map = Convolution(image,kernel,(2,1))      # 2: aloge side rows , 1: along side columns


In [12]:
feature_map.shape

torch.Size([29, 15])

## Adding Padding

In [13]:
def add_padding(image, padding):



  r,c = padding

  if r>0 :
    rows = torch.zeros((r,image.shape[1]))

    image = torch.cat((rows , image , rows),dim=0)

  if c>0:

    columns = torch.zeros((c,image.shape[0]))

    image = torch.cat((columns , image.T , columns),dim=0)


  return image.T



In [14]:
add_padding(torch.randn(2,2),(2,3))

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -0.8560,  0.8522,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -0.1153,  0.8907,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

In [15]:
def Convolution(image,kernel,stride=(1,1),padding = (0,0)):


  a,b = kernel.shape

  r,c = stride

  image_pd = image if padding==(0,0) else add_padding(image,padding)

  feature_map = []

  x,y = image_pd.shape

  for i in range(0,y-b,c):

    row = []

    for j in range(0,x-a,r):

      temp = (image_pd[i:i+b,j:j+a] * kernel).sum(dim=(0,1))

      row.append(temp.item())

    feature_map.append(row)

  return torch.tensor(feature_map)

In [16]:
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)


feature_map = Convolution(image,kernel, padding=(2,2))


In [17]:
feature_map.shape

torch.Size([33, 33])

### Simple adding pad func

In [18]:
def add_padding2(matrix,padding):

  n,m = matrix.shape

  r,c = padding

  padded_matrix = torch.zeros((n+r*2,m+c*2))

  padded_matrix[r:r+n,c:c+m] = matrix

  return padded_matrix

In [19]:
add_padding2(torch.randn(2,2),(2,3))

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -0.5225, -0.3538,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.3305, -0.8458,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

### Adding Dialation

In [117]:
def Convolution(image,kernel,stride=(1,1),padding = (0,0),dilation=(1,1)):


  a,b = kernel.shape

  r,c = stride

  d1,d2 = dilation

  image_pd = image if padding==(0,0) else add_padding2(image,padding)

  feature_map = []

  x,y = image_pd.shape


  for i in range(0,x-a*d1+1,r):

    row = []

    for j in range(0,y-b*d2+1,c):

      temp = (image_pd[i:i+a*d1:d1,j:j+b*d2:d2] * kernel).sum(dim=(0,1))

      row.append(temp.item())

    feature_map.append(row)

  return torch.tensor(feature_map)

In [118]:
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)


feature_map = Convolution(image,kernel,(2,2),(2,2) ,dilation=(2,2))

In [119]:
feature_map.shape

torch.Size([16, 16])

### Test : comparision with pytorch conv2d

In [120]:
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)

out_custom = Convolution(image, kernel, stride=(2,2), padding=(1,1), dilation=(2,2))


img4d = image.unsqueeze(0).unsqueeze(0)       # (1,1,4,4)
kernel4d = kernel.unsqueeze(0).unsqueeze(0) # (1,1,2,2)

out_torch = F.conv2d(img4d, kernel4d, stride=(2,2), padding=(1,1), dilation=(2,2))

print("Custom:\n", out_custom.shape)
print("Torch:\n", out_torch.squeeze().shape)
print("Match? ->", torch.allclose(out_custom,out_torch.squeeze()))

Custom:
 torch.Size([15, 15])
Torch:
 torch.Size([15, 15])
Match? -> True


# All at once

In [124]:
def add_padding2(matrix,padding):
  n,m = matrix.shape
  r,c = padding

  padded_matrix = torch.zeros((n+r*2,m+c*2))
  padded_matrix[r:r+n,c:c+m] = matrix

  return padded_matrix

In [125]:
def Convolution(image, kernel, stride=(1,1), padding=(0,0), dilation=(1,1)):
    a, b = kernel.shape
    r, c = stride
    d1, d2 = dilation

    image_pd = image if padding == (0,0) else add_padding2(image, padding)
    feature_map = []
    x, y = image_pd.shape

    for i in range(0, x - a*d1 + 1, r):
        row = []
        for j in range(0, y - b*d2 + 1, c):

            temp = (image_pd[i:i+a*d1:d1, j:j+b*d2:d2] * kernel).sum(dim=(0,1))
            row.append(temp.item())
        feature_map.append(row)

    return torch.tensor(feature_map)

In [130]:
#torch.manual_seed(220064)
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)

out_custom = Convolution(image, kernel, stride=(2,2), padding=(1,1), dilation=(2,2))


img4d = image.unsqueeze(0).unsqueeze(0)       # (1,1,4,4)
kernel4d = kernel.unsqueeze(0).unsqueeze(0) # (1,1,2,2)

out_torch = F.conv2d(img4d, kernel4d, stride=(2,2), padding=(1,1), dilation=(2,2))

print("Custom:\n", out_custom.shape)
print("Torch:\n", out_torch.squeeze().shape)
print("Match? ->", torch.allclose(out_custom,out_torch.squeeze()))

Custom:
 torch.Size([15, 15])
Torch:
 torch.Size([15, 15])
Match? -> True
